# Analisis exploratorio - Gestión de la Contratación Pública en los Establecimientos Carcelaríos en Colombia

Para fines de este proyecto y de acuerdo con lo que se ha mencionado en anteriores entregas, en principio se tomará los datos de la página del SECOP. En ella, es posible extraer la información relacionada con la gestión de la contratación pública relacionada con el INPEC. 

De acuerdo con lo anterior, filtrando en la base de datos por Nombre Entidad INPEC, es posible extraer un archivo tipo csv la cual contiene 72.520 filas y cuenta con 72 dimensiones. A continuación, se presenta el paso a paso de la limpieza de dichos datos.

Realizando un escaneo general de la forma de la base se encuentra que las dimensiones son: UID, año cargue SECOP, año firma contrato, nivel entidad, orden entidad, nombre entidad, NIT de la entidad, entre otras características relacionadas con los contratistas que han tenido algún contrato con el INPEC. Asimismo, es posible observar que también existen dimensiones relacionadas con la fecha inicio del contrato, fecha fin del contrato tiempo adiciones en meses, plazo de ejecución del contrato, cuantía del contrato, y valor total de adiciones relacionadas con las características del contrato.

**Objetivo:** Ofrecer información de valor al INPEC para la toma de decisiones que impacte positivamente en la administración de su contratación pública. 

**Requerimiento del cliente** 
* Análisis de valor que le permita al INPEC tomar decisiones enfocadas en la mejora de su gestión en la contratación pública.
* Estadísticas relevantes para el entendimiento de la situación contractual actual.
* Información que presente los comportamientos históricos en la contratación pública del INPEC.

In [252]:
# Importacion de librerias
# Tratamiento de datos
# ==========================================
import re
import time
import pandas as pd
import numpy as np
import data_describe as dd
from string import digits
from gensim.utils import simple_preprocess
from gensim.corpora.textcorpus import strip_multiple_whitespaces
# Graficas
# ==========================================
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# API REST
# ==========================================
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

import requests

import warnings
warnings.filterwarnings("ignore")

### Importamos Data - SECOP I 
#### Datos SECOP I

El set de datos de la ``Agencia Nacional de Contratación Pública – Colombia Compra Eficiente`` contiene procesos de compra publica registrados en el ``SECOP I`` desde su implementación, fase de
selección y adjudicación.

|                  Nombre Campo                  |           Tipo          |                                                                                                                                        Descripción                                                                                                                                         |
|:----------------------------------------------:|:-----------------------:|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
| UID                                            | String                  | Identificador único de la relación entre proceso y adjudicación, campo constituido por el número de constancia y el de adjudicación, separados por un guión                                                                                                                                |
| Anno Cargue SECOP                              | Integer                 | Valor Numérico para identificar el año en que el proceso fue cargado en la plataforma                                                                                                                                                                                                      |
| Anno Firma Contrato                            | String                  | Valor de texto que identifica la fecha de firma del contrato, en caso de no existir aún firma del contrato, el registro se marca como no firmado                                                                                                                                           |
| Nivel Entidad                                  | String                  | Caracterización de la entidad de acuerdo con su cobertura, los valores principales son NACIONAL y TERRITORIAL                                                                                                                                                                              |
| Orden Entidad                                  | String                  | Segundo nivel de caracterización, que detalla el orden definido para la entidad                                                                                                                                                                                                            |
| Nombre Entidad                                 | String                  | Nombre de la Entidad del Estado que publicó el proceso de compra, Entidad compradora                                                                                                                                                                                                       |
| NIT de la Entidad                              | String                  | Identificador tributario de la entidad del Estado que publicó el proceso de compra, Entidad compradora                                                                                                                                                                                     |
| Código de la Entidad                           | String                  | Código de la Entidad, utilizado como identificador único en la plataforma SECOP I                                                                                                                                                                                                          |
| ID Modalidad de contratación                   | Integer                 | Identificador de la Modalidad de selección que se determinó para el proceso de compra actual, Ej: Licitación Pública, Contratación Directa.                                                                                                                                                |
| Modalidad de Contratación                      | String                  | Modalidad de selección que se determinó para el proceso de compra actual, Ej: Licitación Pública, Contratación Directa.                                                                                                                                                                    |
| Estado del Proceso                             | String                  | El Estado del proceso a la fecha de publicación                                                                                                                                                                                                                                            |
| Causal de Otras formas de Contratación Directa | String                  | En caso de una contratación directa, el campo determina la causa por la cual se llega a la decisión de contratar directamente.                                                                                                                                                             |
| ID Régimen de Contratación                     | Integer                 | Determina si se utilizan las Leyes 80 de 1993 y 1150 de 2007 o en su defecto un régimen especial                                                                                                                                                                                           |
| Nombre Régimen de Contratación                 | String                  | Determina si se utilizan las Leyes 80 de 1993 y 1150 de 2007 o en su defecto un régimen especial                                                                                                                                                                                           |
| ID Objeto a Contratar                          | String                  | ID del Objeto de la contratación, basado en el catálogo de bienes y servicios UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-de-bienes-yservicios                                                                                                                |
| Objeto a Contratar                             | String                  | Objeto de la contratación, basado en el catálogo de bienes y servicios UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-de-bienes-yservicios                                                                                                                       |
| Detalle del Objeto a Contratar                 | String                  | Adicional al código que define el objeto del contrato, se registra un detalle de la definición del bien o servicio que se adquirirá dentro del proceso                                                                                                                                     |
| Tipo De Contrato                               | String                  | Tipo de Contrato que se realizará, ejemplos: Fiducia, Obra, entre otros.                                                                                                                                                                                                                   |
| Municipio de Obtención                         | String                  | Municipio en el que se desarrolla el proceso de compra pública                                                                                                                                                                                                                             |
| Municipio de Entrega                           | String                  | Municipio en el que se hace la entrega del bien o servicio                                                                                                                                                                                                                                 |
| Municipios Ejecución                           | String                  | Municipios en los que se desarrollará el objeto del proceso de compra pública                                                                                                                                                                                                              |
| Fecha de Cargue en el SECOP                    | Datetime                | Fecha en la que se hizo el registro del proceso de compra en la plataforma, entregada en formato DDMMYYYY                                                                                                                                                                                  |
| Numero de Constancia                           | String                  | Identificador del proceso de compra, generado por SECOP I                                                                                                                                                                                                                                  |
| Numero de Proceso                              | String                  | Identificador del proceso, de acuerdo con la nomenclatura de la entidad                                                                                                                                                                                                                    |
| Numero de Contrato                             | String                  | Identificador del contrato, de acuerdo con la nomenclatura de la entidad                                                                                                                                                                                                                   |
| Cuantía Proceso                                | String                  | Valor por el cual se lanza el proceso de compra                                                                                                                                                                                                                                            |
| ID Grupo                                       | String                  | ID de la Categorización inicial del bien o servicio definido en el proceso de compra, de acuerdo con sus características principales, constituye el primer nivel de clasificación UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-debienes-y-servicios            |
| Nombre Grupo                                   | String                  | Categorización inicial del bien o servicio definido en el proceso de compra, de acuerdo con sus características principales, constituye el primer nivel de clasificación UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-de-bienes-yservicios                     |
| ID Familia                                     | String                  | ID del segundo nivel de la Categorización del bien o servicio definido en el proceso de compra, de acuerdo con sus características principales, constituye el primer nivel de clasificación UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-de-bienes-y-servicios |
| Nombre Familia                                 | String                  | Segundo nivel de la Categorización del bien o servicio definido en el proceso de compra, de acuerdo a sus características principales, constituye el primer nivel de clasificación UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-de-bienes-y-servicios          |
| ID Clase                                       | String                  | ID del tercer nivel de la Categorización del bien o servicio definido en el proceso de compra, de acuerdo a sus características principales, constituye el primer nivel de clasificación UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-de-bienes-y-servicios    |
| Nombre Clase                                   | String                  | Tercer nivel de la Categorización del bien o servicio definido en el proceso de compra, de acuerdo a sus características principales, constituye el primer nivel de clasificación UNSPSC, consultable desde https://www.colombiacompra.gov.co/clasificador-debienes-y-servicios            |
| ID Adjudicación                                | String                  | Identificador de la adjudicación hechas en el proceso de compra                                                                                                                                                                                                                            |
| Tipo Identificación del Contratista            | String                  | Tipo de Identificación del contratista seleccionado en la adjudicación                                                                                                                                                                                                                     |
| Identificación del Contratista                 | String                  | Identificación del contratista seleccionado en la adjudicación                                                                                                                                                                                                                             |
| Nombre Razón Social Contratista                | String                  | Nombre o Razón Social del contratista seleccionado en la adjudicación                                                                                                                                                                                                                      |
| Dpto y Muni Contratista                        | String                  | Departamento y Municipio en el cual opera el contratista seleccionado en la adjudicación                                                                                                                                                                                                   |
| Tipo Documento Representante Legal             | String                  | En caso de que el adjudicado sea una persona jurídica, el tipo de identificación del representante legal de la empresa seleccionada en la adjudicación                                                                                                                                     |
| Identificación Representante Legal             | String                  | En caso de que el adjudicado sea una persona jurídica, registra la identificación del representante legal de la empresa seleccionada en la adjudicación                                                                                                                                    |
| Nombre del Represen Legal                      | String                  | En caso de que el adjudicado sea una persona jurídica, registra el nombre del representante legal de la empresa seleccionada en la adjudicación                                                                                                                                            |
| Fecha de Firma del Contrato                    | Datetime                | Fecha en que se hace la firma del contrato correspondiente a la adjudicación del registro, entregada en formato DDMMYYYY                                                                                                                                                                   |
| Fecha Ini Ejec Contrato                        | Datetime                | Fecha en que inicia la ejecución del contrato, entregada en formato DDMMYYYY                                                                                                                                                                                                               |
| Plazo de Ejec del Contrato                     | String                  | Valor sobre el cual se determina la duración del contrato                                                                                                                                                                                                                                  |
| Rango de Ejec del Contrato                     | String                  | Unidad en la que se define el plazo, pueden ser días, meses o años                                                                                                                                                                                                                         |
| Tiempo Adiciones en Días                       | String                  | Tiempo adicionado al contrato, en días                                                                                                                                                                                                                                                     |
| Tiempo Adiciones en Meses                      | String                  | Tiempo adicionado al contrato, cuando es más de un mes                                                                                                                                                                                                                                     |
| Fecha Fin Ejec Contrato                        | Datetime                | Fecha en que finaliza la ejecución del contrato, entregada en formato DDMMYYYY                                                                                                                                                                                                             |
| Compromiso Presupuestal                        | String                  | En caso de tener registro presupuestal, el campo muestra el código correspondiente                                                                                                                                                                                                         |
| Cuantía Contrato                               | String                  | Valor por el cual se firma el contrato                                                                                                                                                                                                                                                     |
| Valor Total de Adiciones                       | String                  | Valor de la suma de las adiciones hechas al contrato                                                                                                                                                                                                                                       |
| Valor Contrato con Adiciones                   | String                  | Valor Contrato con Adiciones                                                                                                                                                                                                                                                               |
| Objeto del Contrato a la Firma                 | String                  | Valor total del contrato, incluyendo las adicione                                                                                                                                                                                                                                          |
| Proponentes Seleccionados                      | String                  | Listado de los proponentes seleccionados en el proceso de compra                                                                                                                                                                                                                           |
| Calificación Definitiva                        | String                  | Calificación definitiva de los proponentes dentro del proceso de compra                                                                                                                                                                                                                    |
| ID Sub Unidad Ejecutora                        | String                  | Identificador y nombre de la Sub Unidad de ejecución de presupuesto asignada al proceso de compra                                                                                                                                                                                          |
| Nombre Sub Unidad Ejecutora                    | String                  | Identificador y nombre de la Sub Unidad de ejecución de presupuesto asignada al proceso de compra                                                                                                                                                                                          |
| Ruta Proceso en SECOP I                        | String                  | Ruta del proceso de compra en SECOP, para hacer consulta de la información detallada                                                                                                                                                                                                       |
| Moneda                                         | String                  | Moneda en la cual están registradas las cuantías del proceso de compra                                                                                                                                                                                                                     |
| Es PostConflicto                               | Casilla de Verificación | Marcador que indica si el proceso de compra pública está enmarcado en las acciones de postconflicto del acuerdo de paz del año 2017                                                                                                                                                        |
| Marcación Adiciones                            | String                  | Marcación de si el proceso tiene adiciones registradas                                                                                                                                                                                                                                     |
| Posición Rubro                                 | String                  | Posición (Identificador) del Rubro presupuestal                                                                                                                                                                                                                                            |
| Nombre Rubro                                   | String                  | Nombre del Rubro Presupuestal al que corresponde el proceso                                                                                                                                                                                                                                |
| Valor Rubro                                    | Integer                 | Valor del Rubro presupuestal asignado al proceso                                                                                                                                                                                                                                           |
| Sexo Rep Legal                                 | String                  | Sexo del Representante legal de la organización adjudicada                                                                                                                                                                                                                                 |
| Pilar Acuerdo Paz                              | String                  | Pilar, del acuerdo de paz del año 2016, sobre el cual se desarrolla el proceso de compra                                                                                                                                                                                                   |
| Punto Acuerdo Paz                              | String                  | Punto del acuerdo de paz del año 2016, al que se relaciona el objeto de contratación del proceso de compra                                                                                                                                                                                 |
| Municipio Entidad                              | String                  | Municipio en el que se encuentra registrada la entidad estatal compradora                                                                                                                                                                                                                  |
| Departamento Entidad                           | String                  | Departamento en el que se encuentra registrada la entidad estatal compradora                                                                                                                                                                                                               |
| Ultima Actualización                           | Datetime                | Para suplir requerimientos de timestamp, se registra la última fecha de modificación del proceso de compra, entregados en formato DDMMYYYY                                                                                                                                                 |
| Fecha Liquidación                              | Datetime                | Fecha en que fue liquidado el contrato                                                                                                                                                                                                                                                     |

In [2]:
# Define the API endpoint URL
url = "https://www.datos.gov.co/resource/f789-7hwg.json?nombre_entidad=INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO INPEC&$limit=100000"

# Send an HTTP GET request to the API endpoint
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    print("Data received from API:")
else:
    print(f"Request failed with status code {response.status_code}")


Data received from API:


In [117]:
secop_I = pd.DataFrame(data=data)
print(f"Filas:{secop_I.shape[0]}\nColumnas:{secop_I.shape[1]}")
secop_I.head(2)

Filas:72520
Columnas:72


,uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,id_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,fecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,nombre_del_represen_legal,fecha_de_firma_del_contrato,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato,compromiso_presupuestal,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,proponentes_seleccionados,calificacion_definitiva,id_sub_unidad_ejecutora,nombre_sub_unidad_ejecutora,ruta_proceso_en_secop_i,moneda,es_postconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,ultima_actualizacion,cumpledecreto248,incluyebienesdecreto248,fecha_liquidacion
0,17-13-6027483-5540305,2017,2017,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,50000000,"Alimentos, Bebidas y Tabaco",SUMINISTRO DE BEBIDAS COMESTIBLES VIVERES Y RA...,Suministro,Acacías,Acacías,"Acacías, Meta",2017-01-13T00:00:00.000,17-13-6027483,130-17-001,003,54408900,E,[E] Productos de Uso Final,5020,Bebidas,502023,Bebidas no alcohólicas,5540305,Nit de Persona Jurídica,860000258,COMPAÑIA MANUFACTURERA DE PAN COMAPAN,Meta,Cédula de Ciudadanía,41771248,MARÍA FANY RAMIREZ HERNANDEZ,2017-01-20T00:00:00.000,2017-01-20T00:00:00.000,30,D,0,0,2017-02-19T00:00:00.000,10317,3388200,0,3388200,SUMINISTRO DE COMESTIBLES VIVERES Y RANCHO DES...,No Definido,No Definido,37-06-00-100-130,COLONIA AGRÍCOLA DE MÍNIMA SEGURIDAD DE ACACÍAS,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2017-01-19T00:00:00.000,N/D,N/D,NaN
1,17-13-7337888-6713176,2017,2017,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,53000000,"Ropa, Maletas y Productos de Aseo Personal",Contratar la adquisición de elementos de segur...,Compraventa,Fusagasugá,Fusagasugá,"Fusagasugá, Cundinamarca",2017-11-24T00:00:00.000,17-13-7337888,046-2017,043,284191,E,[E] Productos de Uso Final,5310,Ropa,531030,Camisetas,6713176,Nit de Persona Jurídica,900988646,GRUPO INGENIAR SAS,Cundinamarca,Cédula de Ciudadanía,35250339,FLOR MILENA LOZANO CAICEDO,2017-12-30T00:00:00.000,2017-12-07T00:00:00.000,15,D,0,0,2017-12-22T00:00:00.000,76817,270400,0,270400,Contratar la adquisición de elementos de segur...,No Definido,No Definido,12-08-00-119,EPMSC FUSAGASUGÁ,{'url': 'https://www.contratos.gov.co/consulta...,Peso Colombiano,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2017-11-30T00:00:00.000,N/D,N/D,NaN


#### Filtramos data 

1. Filtramos por la columna `estado_del_proceso` los contratos que no se ejecutaron en su finalidad.

In [109]:
pd.DataFrame(secop_I['estado_del_proceso'].value_counts())

,count
estado_del_proceso,
Celebrado,36434
Liquidado,22349
Terminado Anormalmente después de Convocado,5956
Convocado,5433
Terminado sin Liquidar,1635
Borrador,342
Adjudicado,204
Descartado,165
Expresión de Interés,2


* `Celebrado`: el estado "Celebrado" generalmente se refiere a que el contrato ha sido firmado y acordado por ambas partes involucradas.
* `Liquidado`: el estado "Liquidado" hace referencia a que el contrato ha sido terminado y todas las obligaciones y responsabilidades de las partes involucradas han sido cumplidas y cerradas.
* `Terminado Anormalmente después de Convocado`: El estado "Terminado Anormalmente después de Convocado" en el contexto de un proceso o contrato en Colombia sugiere que ha ocurrido una situación inusual o no prevista que ha llevado a la finalización anticipada del proceso o contrato antes de cumplir con todos los requisitos o etapas establecidas
* `Convocado`: En el contexto de un contrato o proceso en Colombia, el estado "Convocado" generalmente se refiere a una etapa en la cual se invita o convoca a las partes interesadas a participar en un proceso o actividad específica. Esta fase suele ser previa al establecimiento de un contrato y puede involucrar la presentación de propuestas, cotizaciones, ofertas, entre otros.
* `Terminado sin Liquidar`: el estado "Terminado sin Liquidar" se refiere a una situación en la que un contrato ha concluido o finalizado, pero aún no se ha llevado a cabo el proceso de liquidación.
* `Borrador`: el estado "Borrador sin Liquidar" en un contrato en Colombia generalmente indica que el contrato aún no ha sido finalizado y está en una etapa preliminar. Un borrador es una versión no definitiva del contrato, que aún puede estar sujeto a cambios, discusiones y negociaciones entre las partes involucradas.
* `Adjudicado`: el estado "Adjudicado sin Liquidar" generalmente se refiere a la situación en la que una entidad ha sido seleccionada como la ganadora de un proceso de contratación, pero el contrato aún no se ha liquidado o cerrado.
* `Descartado`: el estado "Descartado sin Liquidar" en un contrato en Colombia podría referirse a una situación en la que un contrato ha sido anulado o descontinuado, pero aún no se ha llevado a cabo el proceso de liquidación. La liquidación, en términos contractuales, generalmente se refiere a la resolución de las obligaciones pendientes entre las partes involucradas en un contrato, como el pago de deudas, la devolución de bienes o la finalización de servicios.
* `Expresión de Interés`: el término "Expresión de Interés sin Liquidar" no es un término legal estandarizado en Colombia, pero podría referirse a una situación en la que una de las partes en un proceso de contratación o negociación ha expresado su interés en participar o llegar a un acuerdo, pero el proceso no ha sido finalizado o resuelto aún.


Filtramos los contratos que se indice el valor de `Terminado Anormalmente después de Convocado`	y `Expresión de Interés`

In [119]:
anos =["Terminado Anormalmente después de Convocado", "Expresión de Interés"]
secop_I_filtro_estado = secop_I[~secop_I['estado_del_proceso'].isin(anos)]
print(f"Filas:{secop_I_filtro_estado.shape[0]}\nColumnas:{secop_I_filtro_estado.shape[1]}")
secop_I_filtro_estado.head(3)

Filas:66562
Columnas:72


,uid,anno_cargue_secop,anno_firma_contrato,nivel_entidad,orden_entidad,nombre_entidad,nit_de_la_entidad,c_digo_de_la_entidad,id_modalidad,modalidad_de_contratacion,estado_del_proceso,causal_de_otras_formas_de,id_regimen_de_contratacion,nombre_regimen_de_contratacion,id_objeto_a_contratar,objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipio_de_entrega,municipios_ejecucion,fecha_de_cargue_en_el_secop,numero_de_constancia,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,dpto_y_muni_contratista,tipo_doc_representante_legal,identific_representante_legal,nombre_del_represen_legal,fecha_de_firma_del_contrato,fecha_ini_ejec_contrato,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,fecha_fin_ejec_contrato,compromiso_presupuestal,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,proponentes_seleccionados,calificacion_definitiva,id_sub_unidad_ejecutora,nombre_sub_unidad_ejecutora,ruta_proceso_en_secop_i,moneda,es_postconflicto,marcacion_adiciones,posicion_rubro,nombre_rubro,valor_rubro,sexo_replegal,pilar_acuerdo_paz,punto_acuerdo_paz,municipio_entidad,departamento_entidad,ultima_actualizacion,cumpledecreto248,incluyebienesdecreto248,fecha_liquidacion
0,17-13-6027483-5540305,2017,2017,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,50000000,"Alimentos, Bebidas y Tabaco",SUMINISTRO DE BEBIDAS COMESTIBLES VIVERES Y RA...,Suministro,Acacías,Acacías,"Acacías, Meta",2017-01-13T00:00:00.000,17-13-6027483,130-17-001,003,54408900,E,[E] Productos de Uso Final,5020,Bebidas,502023,Bebidas no alcohólicas,5540305,Nit de Persona Jurídica,860000258,COMPAÑIA MANUFACTURERA DE PAN COMAPAN,Meta,Cédula de Ciudadanía,41771248,MARÍA FANY RAMIREZ HERNANDEZ,2017-01-20T00:00:00.000,2017-01-20T00:00:00.000,30,D,0,0,2017-02-19T00:00:00.000,10317,3388200,0,3388200,SUMINISTRO DE COMESTIBLES VIVERES Y RANCHO DES...,No Definido,No Definido,37-06-00-100-130,COLONIA AGRÍCOLA DE MÍNIMA SEGURIDAD DE ACACÍAS,{'url': 'https://www.contratos.gov.co/consulta...,No Definido,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2017-01-19T00:00:00.000,N/D,N/D,NaN
1,17-13-7337888-6713176,2017,2017,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,53000000,"Ropa, Maletas y Productos de Aseo Personal",Contratar la adquisición de elementos de segur...,Compraventa,Fusagasugá,Fusagasugá,"Fusagasugá, Cundinamarca",2017-11-24T00:00:00.000,17-13-7337888,046-2017,043,284191,E,[E] Productos de Uso Final,5310,Ropa,531030,Camisetas,6713176,Nit de Persona Jurídica,900988646,GRUPO INGENIAR SAS,Cundinamarca,Cédula de Ciudadanía,35250339,FLOR MILENA LOZANO CAICEDO,2017-12-30T00:00:00.000,2017-12-07T00:00:00.000,15,D,0,0,2017-12-22T00:00:00.000,76817,270400,0,270400,Contratar la adquisición de elementos de segur...,No Definido,No Definido,12-08-00-119,EPMSC FUSAGASUGÁ,{'url': 'https://www.contratos.gov.co/consulta...,Peso Colombiano,0,0,No Definido,No Definido,0,N,No Definido,No Definido,Bogotá DC,Bogotá DC,2017-11-30T00:00:00.000,N/D,N/D,NaN
2,12-13-859180-822946,2012,2012,NACIONAL,NACIONAL DESCENTRALIZADO,INSTITUTO NACIONAL PENITENCIARIO Y CARCELARIO ...,800215546,112008000,13,Contratación Mínima Cuantía,Celebrado,No Definido,13,Contratación Mínima Cuantía,83000000,Servicios Públicos y Servicios Relacionados co...,RECARGA DE CILINDROS DE GAS DE 100 LIBRAS,Compraventa,San Miguel de Mocoa,San Miguel de Mocoa,"San Miguel de Mocoa, Putumayo",2012-03-26T00:00:00.000,12-13-859180,224

2. Filtrar los datos para solo tres años. Se escogeran los años que tengan mayor data.

In [120]:
pd.DataFrame(secop_I_filtro_estado['anno_cargue_secop'].value_counts())

,count
anno_cargue_secop,
2012,12125
2013,10441
2014,9865
2015,9110
2016,8839
2017,7980
2011,6703
2019,443
2005,236


In [123]:
anos =["2015","2016","2017","2018","2019"]
secop_I_filtro_ano = secop_I_filtro_estado[secop_I_filtro_estado['anno_cargue_secop'].isin(anos)]
pd.DataFrame(secop_I_filtro_ano['anno_cargue_secop'].value_counts())

,count
anno_cargue_secop,
2015,9110
2016,8839
2017,7980
2019,443
2018,181


#### Tipo de cada columna

A continuación podemos observar cada tipo de columna y la memoria que utiliza el proceso.

In [124]:
# Tipo de cada columna
# ==============================================================================
# En pandas, el tipo "object" hace referencia a strings
# datos.dtypes
secop_I_filtro_ano.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26553 entries, 0 to 72517
Data columns (total 72 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   uid                             26553 non-null  object
 1   anno_cargue_secop               26553 non-null  object
 2   anno_firma_contrato             26553 non-null  object
 3   nivel_entidad                   26553 non-null  object
 4   orden_entidad                   26553 non-null  object
 5   nombre_entidad                  26553 non-null  object
 6   nit_de_la_entidad               26553 non-null  object
 7   c_digo_de_la_entidad            26553 non-null  object
 8   id_modalidad                    26553 non-null  object
 9   modalidad_de_contratacion       26553 non-null  object
 10  estado_del_proceso              26553 non-null  object
 11  causal_de_otras_formas_de       26553 non-null  object
 12  id_regimen_de_contratacion      26553 non-null  obj

#### Número de observaciones y valores ausentes

Junto con el estudio del tipo de variables, es básico conocer el número de observaciones disponibles y si todas ellas están completas. Los valores ausentes son muy importantes a la hora de crear modelos, la mayoría de algoritmos no aceptan observaciones incompletas o bien se ven muy influenciados por ellas.

In [125]:
# Dimensiones del dataset
# ==============================================================================
print("Número de filas:", secop_I_filtro_ano.shape[0])
print("Número de columnas:", secop_I_filtro_ano.shape[1])

Número de filas: 26553
Número de columnas: 72


In [126]:
#pd.set_option('display.max_rows', None)
# Número de datos ausentes por variable
# ==============================================================================
pd.DataFrame(secop_I_filtro_ano.isna().sum().sort_values(ascending=False), columns=["Cantidad de Valores Nulos"]).head(13)

,Cantidad de Valores Nulos
fecha_liquidacion,16432
fecha_ini_ejec_contrato,1406
fecha_fin_ejec_contrato,1406
fecha_de_firma_del_contrato,1406
identific_representante_legal,2
calificacion_definitiva,1
nombre_del_represen_legal,1
plazo_de_ejec_del_contrato,0
rango_de_ejec_del_contrato,0
tiempo_adiciones_en_dias,0


En el siguiente dataframe podemos visualizar la cantidad de valores nulos por columna. 
Se observa que lo siguente:
* La columna de ``Fecha Liquidacion`` tiene el 69.26% de los valores como NULL.
* La columna de ``Fecha Fin Ejec Contrato`` tiene el 16.93% de los valores como NULL.
* La columna de ``Fecha de Firma del Contrato`` tiene el 16.68% de los valores como NULL.
* La columna de ``Fecha Ini Ejec Contrato`` tiene el 16.68% de los valores como NULL.
* La columna de ``Calificacion Definitiva`` tiene el 0.22% de los valores como NULL.
* La columna de ``Identific Representante Legal`` tiene el 0.012% de los valores como NULL.


#### Remover columnas innecesarias

Para tener un mejor panorama de los datos se realizo una entrevista a un contratista vigente del estado (Mario Rivera), el cual asesoro acerca de contratación publica y determino las variables mas importantes a tener en cuenta para la detección de patrones de contratación, adiciones y ejecuciones que ayude con el objetivo de  ofrecer información de valor al INPEC para la toma de decisiones que impacte positivamente en la administración de su contratación pública.

A continuación se eliminaran las columnas que no sean representativas para el requerimiento del cliente.

In [127]:
secop_I_filter = secop_I_filtro_ano[['uid', 'anno_cargue_secop','id_modalidad', 'modalidad_de_contratacion',
                          'estado_del_proceso', 'id_objeto_a_contratar',
                          'detalle_del_objeto_a_contratar', 'tipo_de_contrato', 'municipio_de_obtencion',
                          'municipios_ejecucion','numero_de_proceso', 'numero_de_contrato', 'cuantia_proceso',
                          'id_grupo', 'nombre_grupo', 'id_familia','nombre_familia', 'id_clase', 'nombre_clase',
                          'id_adjudicacion','tipo_identifi_del_contratista', 'identificacion_del_contratista',
                          'nom_razon_social_contratista','tipo_doc_representante_legal', 'identific_representante_legal',
                          'plazo_de_ejec_del_contrato', 'rango_de_ejec_del_contrato', 'tiempo_adiciones_en_dias','tiempo_adiciones_en_meses',
                          'cuantia_contrato', 'valor_total_de_adiciones', 'valor_contrato_con_adiciones','objeto_del_contrato_a_la',
                          'ruta_proceso_en_secop_i','moneda'
                          ]]
secop_I_filter.head(2)

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato,rango_de_ejec_del_contrato,tiempo_adiciones_en_dias,tiempo_adiciones_en_meses,cuantia_contrato,valor_total_de_adiciones,valor_contrato_con_adiciones,objeto_del_contrato_a_la,ruta_proceso_en_secop_i,moneda
0,17-13-6027483-5540305,2017,13,Contratación Mínima Cuantía,Celebrado,50000000,SUMINISTRO DE BEBIDAS COMESTIBLES VIVERES Y RA...,Suministro,Acacías,"Acacías, Meta",130-17-001,003,54408900,E,[E] Productos de Uso Final,5020,Bebidas,502023,Bebidas no alcohólicas,5540305,Nit de Persona Jurídica,860000258,COMPAÑIA MANUFACTURERA DE PAN COMAPAN,Cédula de Ciudadanía,41771248,30,D,0,0,3388200,0,3388200,SUMINISTRO DE COMESTIBLES VIVERES Y RANCHO DES...,{'url': 'https://www.contratos.gov.co/consulta...,No Definido
1,17-13-7337888-6713176,2017,13,Contratación Mínima Cuantía,Celebrado,53000000,Contratar la adquisición de elementos de segur...,Compraventa,Fusagasugá,"Fusagasugá, Cundinamarca",046-2017,043,284191,E,[E] Productos de Uso Final,5310,Ropa,531030,Camisetas,6713176,Nit de Persona Jurídica,900988646,GRUPO INGENIAR SAS,Cédula de Ciudadanía,35250339,15,D,0,0,270400,0,270400,Contratar la adquisición de elementos de segur...,{'url': 'https://www.contratos.gov.co/consulta...,Peso Colombiano


### Normalización de los datos

* Elimino "-" de las columnas **uid** y **numero_de_proceso**, con la finalidad de eliminar caracteris y dejar la forma númerica de las columnas

In [128]:
# Elimino "-" de todos los uid
secop_I_filter['uid'] = secop_I_filter['uid'].apply(lambda x: x.replace('-', '') if isinstance(x, str) else x)
secop_I_filter['numero_de_proceso'] = secop_I_filter['numero_de_proceso'].apply(lambda x: x.replace('-', '') if isinstance(x, str) else x)

* Elimino espacios de la columna **numero_de_contrato** en la data

In [129]:
# Elimino espacios en la data
secop_I_filter['numero_de_contrato'] = secop_I_filter['numero_de_contrato'].str.replace(r'\s', '', regex=True)

* Se Separo **Municipios** y **Departamento** de una misma columna con la finalidad de segementar optimamente los datos en dos diferentes columnas

In [130]:
# Separo municipio y departamento
split_columns = secop_I_filter['municipios_ejecucion'].str.split(',', expand=True)
data_mun_dep = split_columns.loc[:,:1]
data_mun_dep.columns = ['municipios_ejecucion_', 'departamento_ejecucion']
secop_I_filter['municipios_ejecucion_'] = data_mun_dep.municipios_ejecucion_.to_list()
secop_I_filter['departamento_ejecucion'] = data_mun_dep.departamento_ejecucion.to_list()
## organizo las columhns de municipio o departamento
secop_I_filter = secop_I_filter[['uid', 'anno_cargue_secop', 'id_modalidad', 'modalidad_de_contratacion',
       'estado_del_proceso', 'id_objeto_a_contratar',
       'detalle_del_objeto_a_contratar', 'tipo_de_contrato',
       'municipio_de_obtencion', 'municipios_ejecucion_',
       'departamento_ejecucion','numero_de_proceso',
       'numero_de_contrato', 'cuantia_proceso', 'id_grupo', 'nombre_grupo',
       'id_familia', 'nombre_familia', 'id_clase', 'nombre_clase',
       'id_adjudicacion', 'tipo_identifi_del_contratista',
       'identificacion_del_contratista', 'nom_razon_social_contratista',
       'tipo_doc_representante_legal', 'identific_representante_legal',
       'plazo_de_ejec_del_contrato', 'rango_de_ejec_del_contrato',
       'tiempo_adiciones_en_dias', 'tiempo_adiciones_en_meses',
       'cuantia_contrato', 'valor_total_de_adiciones',
       'valor_contrato_con_adiciones','moneda','objeto_del_contrato_a_la',
       'ruta_proceso_en_secop_i']]

*  Para las columnas de **plazo_de_ejec_del_contrato** y **rango_de_ejec_del_contrato** se normaliza en un tiempo de días unicamente. 
Dado que la duración real de un mes puede variar, un enfoque común es utilizar la duración media de 30,44 días.


In [131]:
# Función para convertir el dataframe 
def convert_months_to_days(row):
    if row['rango_de_ejec_del_contrato'] == "M":
        average_days_per_month = 30 #30.44
        days = int(row['plazo_de_ejec_del_contrato']) * average_days_per_month
        return days
    return row['plazo_de_ejec_del_contrato']

# Convierto la columna en tipo INTEGER
secop_I_filter['plazo_de_ejec_del_contrato'] = secop_I_filter['plazo_de_ejec_del_contrato'].astype(int)
secop_I_filter['plazo_de_ejec_del_contrato_'] = secop_I_filter.apply(convert_months_to_days, axis=1)

# Remplazo rango_de_ejec_del_contrato por dias en todas las filas
secop_I_filter['rango_de_ejec_del_contrato'] = secop_I_filter['rango_de_ejec_del_contrato'].replace('M', 'D')

# Organizo el dataframe con los resultados
secop_I_filter_norm = secop_I_filter[['uid', 'anno_cargue_secop', 'id_modalidad', 'modalidad_de_contratacion',
       'estado_del_proceso', 'id_objeto_a_contratar',
       'detalle_del_objeto_a_contratar', 'tipo_de_contrato',
       'municipio_de_obtencion', 'municipios_ejecucion_',
       'departamento_ejecucion', 'numero_de_proceso', 'numero_de_contrato',
       'cuantia_proceso', 'id_grupo', 'nombre_grupo', 'id_familia',
       'nombre_familia', 'id_clase', 'nombre_clase', 'id_adjudicacion',
       'tipo_identifi_del_contratista', 'identificacion_del_contratista',
       'nom_razon_social_contratista', 'tipo_doc_representante_legal',
       'identific_representante_legal', 'plazo_de_ejec_del_contrato_',
       'rango_de_ejec_del_contrato', 'tiempo_adiciones_en_dias', 'cuantia_contrato',
       'valor_total_de_adiciones', 'valor_contrato_con_adiciones', 'moneda',
       'objeto_del_contrato_a_la', 'ruta_proceso_en_secop_i']]

* Se define la columna de PRÓRROGADO

In [132]:
# Función para convertir el dataframe 
secop_I_filter_norm['tiempo_adiciones_en_dias'] = secop_I_filter_norm['tiempo_adiciones_en_dias'].astype(int)
secop_I_filter_norm['prorrogado'] = np.where(secop_I_filter_norm['tiempo_adiciones_en_dias'] != 0, 1, 0)
# Organizo dataframe despues de la respuesta
secop_I_filter_norm = secop_I_filter_norm[['uid', 'anno_cargue_secop', 'id_modalidad', 'modalidad_de_contratacion',
       'estado_del_proceso', 'id_objeto_a_contratar',
       'detalle_del_objeto_a_contratar', 'tipo_de_contrato',
       'municipio_de_obtencion', 'municipios_ejecucion_',
       'departamento_ejecucion', 'numero_de_proceso', 'numero_de_contrato',
       'cuantia_proceso', 'id_grupo', 'nombre_grupo', 'id_familia',
       'nombre_familia', 'id_clase', 'nombre_clase', 'id_adjudicacion',
       'tipo_identifi_del_contratista', 'identificacion_del_contratista',
       'nom_razon_social_contratista', 'tipo_doc_representante_legal',
       'identific_representante_legal', 'plazo_de_ejec_del_contrato_',
       'rango_de_ejec_del_contrato','prorrogado','tiempo_adiciones_en_dias',
       'cuantia_contrato', 'valor_total_de_adiciones',
       'valor_contrato_con_adiciones', 'moneda', 'objeto_del_contrato_a_la',
       'ruta_proceso_en_secop_i']]

* Se define la columna de CON ADICIÓN PRESUPUESTAL

In [133]:
secop_I_filter_norm["valor_total_de_adiciones"] = secop_I_filter_norm["valor_total_de_adiciones"].astype(int)
secop_I_filter_norm['con_adicion_pptal'] = np.where(secop_I_filter_norm['valor_total_de_adiciones'] != 0, 1, 0)
secop_I_filter_norm = secop_I_filter_norm[['uid', 'anno_cargue_secop', 'id_modalidad', 'modalidad_de_contratacion',
       'estado_del_proceso', 'id_objeto_a_contratar',
       'detalle_del_objeto_a_contratar', 'tipo_de_contrato',
       'municipio_de_obtencion', 'municipios_ejecucion_',
       'departamento_ejecucion', 'numero_de_proceso', 'numero_de_contrato',
       'cuantia_proceso', 'id_grupo', 'nombre_grupo', 'id_familia',
       'nombre_familia', 'id_clase', 'nombre_clase', 'id_adjudicacion',
       'tipo_identifi_del_contratista', 'identificacion_del_contratista',
       'nom_razon_social_contratista', 'tipo_doc_representante_legal',
       'identific_representante_legal', 'plazo_de_ejec_del_contrato_',
       'rango_de_ejec_del_contrato', 'prorrogado', 'tiempo_adiciones_en_dias',
       'cuantia_contrato', 'con_adicion_pptal' ,'valor_total_de_adiciones',
       'valor_contrato_con_adiciones', 'moneda', 'objeto_del_contrato_a_la',
       'ruta_proceso_en_secop_i']]

secop_I_filter_norm.reset_index(drop=True, inplace=True)
secop_I_filter_norm.head(2)

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,171360274835540305,2017,13,Contratación Mínima Cuantía,Celebrado,50000000,SUMINISTRO DE BEBIDAS COMESTIBLES VIVERES Y RA...,Suministro,Acacías,Acacías,Meta,13017001,003,54408900,E,[E] Productos de Uso Final,5020,Bebidas,502023,Bebidas no alcohólicas,5540305,Nit de Persona Jurídica,860000258,COMPAÑIA MANUFACTURERA DE PAN COMAPAN,Cédula de Ciudadanía,41771248,30,D,0,0,3388200,0,0,3388200,No Definido,SUMINISTRO DE COMESTIBLES VIVERES Y RANCHO DES...,{'url': 'https://www.contratos.gov.co/consulta...
1,171373378886713176,2017,13,Contratación Mínima Cuantía,Celebrado,53000000,Contratar la adquisición de elementos de segur...,Compraventa,Fusagasugá,Fusagasugá,Cundinamarca,0462017,043,284191,E,[E] Productos de Uso Final,5310,Ropa,531030,Camisetas,6713176,Nit de Persona Jurídica,900988646,GRUPO INGENIAR SAS,Cédula de Ciudadanía,35250339,15,D,0,0,270400,0,0,270400,Peso Colombiano,Contratar la adquisición de elementos de segur...,{'url': 'https://www.contratos.gov.co/consulta...


### Relleno de data incompleta - `No Definida`

Por medio de tecnica scramble html se puede acceder a la Hoja del proceso de cada contrato con la finalidad de obtener información faltante de la base de datos y asi poder completar información de valor para el analisis.

Esta base de datos define estos datos faltante como valore ``No Definido``, la idea es rellenar estos valores con los extraidos de la tecnica de scramble

In [194]:
def found_miss_data(dataframe:pd.DataFrame, valor_especifico:str):
    # Establecer el valor específico que deseas encontrar
    valor_especifico = valor_especifico
    # Utilizar el método 'isin()' para encontrar las columnas que contienen el valor específico
    columnas_con_valor_especifico = dataframe.isin([valor_especifico]).any()
    # Filtrar las columnas que contienen el valor específico
    columnas_filtradas = columnas_con_valor_especifico[columnas_con_valor_especifico].index.tolist()
    # Contar la cantidad de veces que se repite el valor específico en cada columna
    specific_value_counts = pd.DataFrame(dataframe.apply(lambda x: (x == valor_especifico).sum()), columns=["count"])

    return specific_value_counts, columnas_filtradas

specific_value_counts, columnas_filtradas = found_miss_data(secop_I_filter_norm, 'No Definido')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 11


,count
tipo_de_contrato,1
municipio_de_obtencion,248
tipo_identifi_del_contratista,1406
identificacion_del_contratista,1406
nom_razon_social_contratista,1406
tipo_doc_representante_legal,1406
identific_representante_legal,1406
objeto_del_contrato_a_la,1406
nombre_familia,4066
nombre_clase,4066


Estas son las columnas que hacen falta por datos y que a continuación se extraeran los datos en la hoja de detalle del proceso con la finalidad de completar la información

In [142]:
# Dataframe con data No Definida
secop_I_filter_norm_scra = secop_I_filter_norm.copy()
secop_I_filter_norm_scra.head(3)

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,171360274835540305,2017,13,Contratación Mínima Cuantía,Celebrado,50000000,SUMINISTRO DE BEBIDAS COMESTIBLES VIVERES Y RA...,Suministro,Acacías,Acacías,Meta,13017001,003,54408900,E,[E] Productos de Uso Final,5020,Bebidas,502023,Bebidas no alcohólicas,5540305,Nit de Persona Jurídica,860000258,COMPAÑIA MANUFACTURERA DE PAN COMAPAN,Cédula de Ciudadanía,41771248,30,D,0,0,3388200,0,0,3388200,No Definido,SUMINISTRO DE COMESTIBLES VIVERES Y RANCHO DES...,{'url': 'https://www.contratos.gov.co/consulta...
1,171373378886713176,2017,13,Contratación Mínima Cuantía,Celebrado,53000000,Contratar la adquisición de elementos de segur...,Compraventa,Fusagasugá,Fusagasugá,Cundinamarca,0462017,043,284191,E,[E] Productos de Uso Final,5310,Ropa,531030,Camisetas,6713176,Nit de Persona Jurídica,900988646,GRUPO INGENIAR SAS,Cédula de Ciudadanía,35250339,15,D,0,0,270400,0,0,270400,Peso Colombiano,Contratar la adquisición de elementos de segur...,{'url': 'https://www.contratos.gov.co/consulta...
2,151341535123918724,2015,13,Contratación Mínima Cuantía,Celebrado,53000000,Adquisición de kits de elementos de aseo perso...,Compraventa,Bucaramanga,Bucaramanga,Santander,410 043 DE 2515,AceptaciondeofertacontratoN042de2015,52875000,E,[E] Productos de Uso Final,5313,Artículos de tocador y cuidado personal,531316,Baño y cuerpo,3918724,Nit de Persona Jurídica,8040037441,CENTRAL DE SUMINISTROS LTDA,Cédula de Ciudadanía,28212971 de Lebrija,30,D,0,0,52872736,0,0,52872736,No Definido,Adquisición de kits de elementos de aseo perso...,{'url': 'https://www.contratos.gov.co/consulta...


In [163]:
# Fetch the web page
url = secop_I_filter_norm['ruta_proceso_en_secop_i'][274]['url'] 
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Extract information using BeautifulSoup 
    paragraphs = soup.find_all('tr')
    result = []
    for row in paragraphs:
        cells = row.find_all("td")
        cell_data = [cell.text.strip() for cell in cells]
        if "Detalle y Cantidad del Objeto a Contratar"  in cell_data:
            result.append(cell_data[1])  

result[0]

'CONTRATACIÓN PARA EL SUMINISTRO DE ALIMENTOS PROCESADOS, EMPANADAS, PAPAS Y AREPAS RELLENAS PARA EL EXPENDIÓ DEL EPMSCRIOHACHA.'

In [87]:
secop_I_filter_norm['ruta_proceso_en_secop_i'][0]['url'] 

'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=17-13-6027483'

In [155]:

def extract_info_cuantia_url_2(url:str, columna:str):

    # Instalamos la version de chromedriver correspondiente
    ruta_chromedriver = ChromeDriverManager(path='./chromedriver').install()
    # instanciamos al servicio chromedriver
    s = Service(ruta_chromedriver)
    # instalamos web driver de selenium con chrome
    driver = webdriver.Chrome(service=s)
    # Url de petición y abrimos la url
    
    driver.get(url=url)
    
    # Parse the HTML content
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract information using BeautifulSoup 
    paragraphs = soup.find_all('tr')
    result = []
    if columna == "Cuantía Definitiva del Contrato":
        for row in paragraphs:
            cells = row.find_all("td")
            cell_data = [cell.text.strip() for cell in cells]
            if "Cuantía Definitiva del Contrato" in cell_data:
                # Separate the number and the text using regex
                valor_contrato = float(re.findall(r'[\d,]+(?:\.\d+)?', cell_data[1])[0].replace(',', ''))
                text = re.findall(r'[^\d,.]+', cell_data[1])
                # Removing leading and trailing whitespaces from the text
                moneda = ''.join([t.strip() for t in text]).replace('$', '')
                cell_data[1] = int(valor_contrato)
                cell_data.append(moneda)
                result.append(cell_data)
    
        return result
    # Para extraer data de Tipo de Contrato
    if columna == "Tipo de Contrato":
        for row in paragraphs:
            cells = row.find_all("td")
            cell_data = [cell.text.strip() for cell in cells]
            if "Tipo de Contrato" in cell_data:
                result.extend(cell_data)
        return result
    # Para extraer data de Departamento y Municipio    
    if columna == "Departamento y Municipio de Obtención de Documentos":
        for row in paragraphs:
            cells = row.find_all("td")
            cell_data = [cell.text.strip() for cell in cells]
            if "Departamento y Municipio de Obtención de Documentos" in cell_data:
                resultado = cell_data[1].split(':')
                result.extend([r.strip() for r in resultado])
        return result
            # if "País y Departamento/Provincia de ubicación del Contratista" in cell_data:
            #     resultado = cell_data[1].split(':')
            #     result.extend([r.strip() for r in resultado])  

    if  columna == "Identificación del Contratista":   
        for row in paragraphs:
            cells = row.find_all("td")
            cell_data = [cell.text.strip() for cell in cells]
            if "Identificación del Contratista"  in cell_data:
                resultado = cell_data[1].split("No", 1)
                result.extend([r.strip() for r in resultado])  
        return result
    
    if  columna == "Detalle y Cantidad del Objeto a Contratar":   
        for row in paragraphs:
            cells = row.find_all("td")
            cell_data = [cell.text.strip() for cell in cells]
            if "Detalle y Cantidad del Objeto a Contratar" in cell_data:
                result.append(cell_data[1])  
        return result

Filtramos los datos de la columna ``tipo_de_contrato=='No Definido'`` y asignamos el valor real obtenido de la Hoja del detalle del proceso de los contratos obtenida de la url de la columna de ``ruta_proceso_en_secop_i``

In [143]:
filtro_tipo_contrato = secop_I_filter_norm_scra[secop_I_filter_norm_scra['tipo_de_contrato']=='No Definido']
print(f"Filas:{filtro_tipo_contrato.shape[0]}\nColumnas:{filtro_tipo_contrato.shape[1]}")

Filas:1
Columnas:37


In [144]:
for index, row in filtro_tipo_contrato.iterrows():
    print("Se corrigio el index:",index)
    url = row['ruta_proceso_en_secop_i']['url']
    result = extract_info_cuantia_url_2(url=url, columna="Tipo de Contrato")
    if len(result) != 0:
        secop_I_filter_norm_scra.loc[index, 'tipo_de_contrato'] = result[1]
    else:
        secop_I_filter_norm_scra.loc[index, 'tipo_de_contrato'] = 'No encontrado en Hoja Proceso'

Se corrigio el index: 10540


In [145]:
secop_I_filter_norm_scra.loc[[10540],:"tipo_de_contrato"]

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato
10540,191988590538059861,2019,19,Asociación Público Privada,Celebrado,73000000,LA DESINTEGRACIÓN DE LOS VEHÍCULOS AUTOMOTORES...,No encontrado en Hoja Proceso


Filtramos los datos de la columna ``municipio_de_obtencion=='No Definido'`` y asignamos el valor real obtenido de la Hoja del detalle del proceso de los contratos obtenida de la url de la columna de ``ruta_proceso_en_secop_i``

In [149]:
filtro_municipio_de_obtencion = secop_I_filter_norm_scra[secop_I_filter_norm_scra['municipio_de_obtencion']=='No Definido']
print(f"Filas:{filtro_municipio_de_obtencion.shape[0]}\nColumnas:{filtro_municipio_de_obtencion.shape[1]}")

Filas:248
Columnas:37


In [150]:
index_list = []
for index, row in filtro_municipio_de_obtencion.iterrows():
    url = row['ruta_proceso_en_secop_i']['url']
    result = extract_info_cuantia_url_2(url=url, columna="Departamento y Municipio de Obtención de Documentos")
    index_list.append(index)
    print("Se corrigio el index:",index)
    #print("resultado:", result)
    if len(result) != 0:
        secop_I_filter_norm_scra.loc[index, 'municipio_de_obtencion'] = result[1]
    else:
        secop_I_filter_norm_scra.loc[index, 'municipio_de_obtencion'] = 'No encontrado en Hoja Proceso'
    time.sleep(5)

Se corrigio el index: 19
Se corrigio el index: 42
Se corrigio el index: 281
Se corrigio el index: 449
Se corrigio el index: 566
Se corrigio el index: 570
Se corrigio el index: 1043
Se corrigio el index: 1136
Se corrigio el index: 1387
Se corrigio el index: 1390
Se corrigio el index: 1552
Se corrigio el index: 1577
Se corrigio el index: 1703
Se corrigio el index: 1705
Se corrigio el index: 2078
Se corrigio el index: 2418
Se corrigio el index: 2473
Se corrigio el index: 2551
Se corrigio el index: 2567
Se corrigio el index: 2581
Se corrigio el index: 2798
Se corrigio el index: 2836
Se corrigio el index: 3023
Se corrigio el index: 3079
Se corrigio el index: 3249
Se corrigio el index: 3587
Se corrigio el index: 3673
Se corrigio el index: 3723
Se corrigio el index: 3834
Se corrigio el index: 3894
Se corrigio el index: 4159
Se corrigio el index: 4321
Se corrigio el index: 4334
Se corrigio el index: 4432
Se corrigio el index: 4440
Se corrigio el index: 4733
Se corrigio el index: 4778
Se corrig

In [152]:
secop_I_filter_norm_scra.loc[index_list,"municipio_de_obtencion"]

19       No encontrado en Hoja Proceso
42       No encontrado en Hoja Proceso
281      No encontrado en Hoja Proceso
449      No encontrado en Hoja Proceso
566      No encontrado en Hoja Proceso
                     ...              
25646    No encontrado en Hoja Proceso
25681    No encontrado en Hoja Proceso
25811    No encontrado en Hoja Proceso
26112    No encontrado en Hoja Proceso
26345    No encontrado en Hoja Proceso
Name: municipio_de_obtencion, Length: 248, dtype: object

Filtramos los datos de la columna ``objeto_del_contrato_a_la=='No Definido'`` y asignamos el valor real obtenido de la Hoja del detalle del proceso de los contratos obtenida de la url de la columna de ``ruta_proceso_en_secop_i``

In [201]:
filtro_objeto_contrato = secop_I_filter_norm_scra[secop_I_filter_norm_scra['objeto_del_contrato_a_la']=='No Definido']
print(f"Filas:{filtro_objeto_contrato.shape[0]}\nColumnas:{filtro_objeto_contrato.shape[1]}")

Filas:1
Columnas:37


In [202]:
index_list = []
for index, row in filtro_objeto_contrato.iterrows():
    url = row['ruta_proceso_en_secop_i']['url']
    result = extract_info_cuantia_url_2(url=url, columna="Detalle y Cantidad del Objeto a Contratar")
    index_list.append(index)
    print("Se corrigio el index:",index)
    print("resultado:", result)
    if len(result) != 0:
        secop_I_filter_norm_scra.loc[index, 'objeto_del_contrato_a_la'] = result[0]
    else:
        secop_I_filter_norm_scra.loc[index, 'objeto_del_contrato_a_la'] = 'No encontrado Hoja Proceso'
    time.sleep(5)

Se corrigio el index: 469
resultado: ['CONTRATAR LA COMPRA DE BATIDORA MULTIPROPÓSITO PARA EL PROYECTO PRODUCTIVO DE PANADERÍA DEL ESTABLECIMIENTO']


In [180]:
secop_I_filter_norm_scra.loc[index_list,"objeto_del_contrato_a_la"]

22123    contratar los servicios de transporte para el ...
22654    ADQUISICION DE GASOLINA PARA LOS VEHICULOS DEL...
Name: objeto_del_contrato_a_la, dtype: object

In [182]:
secop_I_filter_norm_scra.to_csv('data/extrac_II.csv', index=False)


Filtramos los datos de la columna ``moneda=='No Definido'`` y asignamos el valor real obtenido de la Hoja del detalle del proceso de los contratos obtenida de la url de la columna de ``ruta_proceso_en_secop_i``

In [199]:
# Dataframe con data No Definida
secol_I_filter_moneda = secop_I_filter_norm_scra[secop_I_filter_norm_scra['moneda']=='No Definido']
print(f"Filas:{secol_I_filter_moneda.shape[0]}\nColumnas:{secol_I_filter_moneda.shape[1]}")

Filas:0
Columnas:37


In [190]:
index_list = []
for index, row in secol_I_filter_moneda[1046:].iterrows():
    url = row['ruta_proceso_en_secop_i']['url']
    result = extract_info_cuantia_url_2(url=url, columna="Cuantía Definitiva del Contrato")
    index_list.append(index)
    print("Se corrigio el index:",index)
    print("resultado:", result)
    if len(result) != 0:
        secop_I_filter_norm_scra.loc[index,'moneda'] =  result[0][2] if result else None
        # secop_I_filter_norm_scra.loc[index, 'cuantia_definitiva_contratos_asoc']=sum(r[1] for r in result)
    else:
        secop_I_filter_norm_scra.loc[index, 'moneda'] = 'No encontrado Hoja Proceso'
    time.sleep(5)

### Data incompleta - `N/D`

Por medio de tecnica scramble html se puede acceder a la Hoja del proceso de cada contrato con la finalidad de obtener información faltante de la base de datos y asi poder completar información de valor para el analisis.

Esta base de datos define estos datos faltante como valore ``N/D``, la idea es rellenar estos valores con los extraidos de la tecnica de scramble

In [236]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_I_filter_norm_scra, 'N/D')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 2


,count
id_familia,4066
id_clase,4066


In [243]:
secol_I_filter_id_clase = secop_I_filter_norm_scra[secop_I_filter_norm_scra['id_clase']=='N/D']
secol_I_filter_id_clase['ruta_proceso_en_secop_i'].to_list()[0:10]


[{'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3473508'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-12-3375990'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3469541'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3306680'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-12-3356925'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3827149'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3915164'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3306203'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3584881'},
 {'url': 'https://www.contratos.gov.co/consultas/detalleProceso.do?numConstancia=15-13-3588702'}]

Estos valores en `'N/D'` aparecen en contratos de años <2015, es decir que antes solo se tenia en cuenta el `grupo` y `segmento` al que pertenecia el contrato y despues le añadieron los valores de `familia` y `clase`.

#### Verifico relleno de data

In [203]:
specific_value_counts, columnas_filtradas = found_miss_data(secop_I_filter_norm_scra, 'No Definido')
print("\nNúmero de columnas con valores [No Definido]:",len(columnas_filtradas))
specific_value_counts[specific_value_counts["count"] != 0].sort_values(by=['count'])


Número de columnas con valores [No Definido]: 7


,count
tipo_identifi_del_contratista,1406
identificacion_del_contratista,1406
nom_razon_social_contratista,1406
tipo_doc_representante_legal,1406
identific_representante_legal,1406
nombre_familia,4066
nombre_clase,4066


Las siguientes filas quedan pendiente de extracción debido a su poco aporte en el desarrollo del proyecto

#### Definición DTYPES

Se define un tipo de dato para cada columna

In [204]:
secop_I_filter_norm_scra.dtypes

uid                               object
anno_cargue_secop                 object
id_modalidad                      object
modalidad_de_contratacion         object
estado_del_proceso                object
id_objeto_a_contratar             object
detalle_del_objeto_a_contratar    object
tipo_de_contrato                  object
municipio_de_obtencion            object
municipios_ejecucion_             object
departamento_ejecucion            object
numero_de_proceso                 object
numero_de_contrato                object
cuantia_proceso                   object
id_grupo                          object
nombre_grupo                      object
id_familia                        object
nombre_familia                    object
id_clase                          object
nombre_clase                      object
id_adjudicacion                   object
tipo_identifi_del_contratista     object
identificacion_del_contratista    object
nom_razon_social_contratista      object
tipo_doc_represe

In [235]:
secop_I_filter_norm_scra[['anno_cargue_secop', 'id_modalidad','uid']] = secop_I_filter_norm_scra[['anno_cargue_secop', 'id_modalidad','uid']].astype(int)
secop_I_filter_norm_scra[['id_objeto_a_contratar','numero_de_contrato' ]] = secop_I_filter_norm_scra[['id_objeto_a_contratar','numero_de_contrato']].astype(int)
secop_I_filter_norm_scra[['cuantia_contrato','plazo_de_ejec_del_contrato_']] = secop_I_filter_norm_scra[['cuantia_contrato','plazo_de_ejec_del_contrato_']].astype(int)
secop_I_filter_norm_scra[['valor_total_de_adiciones','valor_contrato_con_adiciones']] = secop_I_filter_norm_scra[['valor_total_de_adiciones','valor_contrato_con_adiciones']].astype(int)


In [233]:
secop_I_filter_norm_scra.dtypes

uid                                int64
anno_cargue_secop                  int64
id_modalidad                       int64
modalidad_de_contratacion         object
estado_del_proceso                object
id_objeto_a_contratar              int64
detalle_del_objeto_a_contratar    object
tipo_de_contrato                  object
municipio_de_obtencion            object
municipios_ejecucion_             object
departamento_ejecucion            object
numero_de_proceso                 object
numero_de_contrato                 int64
cuantia_proceso                   object
id_grupo                          object
nombre_grupo                      object
id_familia                        object
nombre_familia                    object
id_clase                          object
nombre_clase                      object
id_adjudicacion                   object
tipo_identifi_del_contratista     object
identificacion_del_contratista    object
nom_razon_social_contratista      object
tipo_doc_represe

#### Normalización Texto

* Se Limpia y normaliza Texto en la data

In [254]:
def norm_text(texto:str):
    word_tokens = simple_preprocess(texto, deacc=True, min_len=2, max_len=40)
    texto = ' '.join(word_tokens)
    return texto

In [259]:
# Check type str
secop_I_filter_norm_scra[['modalidad_de_contratacion', 'estado_del_proceso']] = secop_I_filter_norm_scra[['modalidad_de_contratacion', 'estado_del_proceso']].astype(str)
secop_I_filter_norm_scra[['tipo_de_contrato', 'municipio_de_obtencion']] = secop_I_filter_norm_scra[['tipo_de_contrato', 'municipio_de_obtencion']].astype(str)
secop_I_filter_norm_scra[['municipios_ejecucion_', 'departamento_ejecucion']] = secop_I_filter_norm_scra[['municipios_ejecucion_', 'departamento_ejecucion']].astype(str)
secop_I_filter_norm_scra[['nombre_grupo', 'nombre_familia']] = secop_I_filter_norm_scra[['nombre_grupo','nombre_familia']].astype(str)
secop_I_filter_norm_scra[['nombre_clase', 'tipo_identifi_del_contratista']] = secop_I_filter_norm_scra[['nombre_clase', 'tipo_identifi_del_contratista']].astype(str)
secop_I_filter_norm_scra[['nom_razon_social_contratista','tipo_doc_representante_legal','moneda']] = secop_I_filter_norm_scra[['nom_razon_social_contratista', 'tipo_doc_representante_legal','moneda']].astype(str)

In [260]:
# Apply funtion
secop_I_filter_norm_scra[['modalidad_de_contratacion', 'estado_del_proceso']] = secop_I_filter_norm_scra[['modalidad_de_contratacion', 'estado_del_proceso']].apply(lambda x: x.apply(norm_text))
secop_I_filter_norm_scra[['tipo_de_contrato', 'municipio_de_obtencion']] = secop_I_filter_norm_scra[['tipo_de_contrato', 'municipio_de_obtencion']].apply(lambda x: x.apply(norm_text))
secop_I_filter_norm_scra[['municipios_ejecucion_', 'departamento_ejecucion']] = secop_I_filter_norm_scra[['municipios_ejecucion_', 'departamento_ejecucion']].apply(lambda x: x.apply(norm_text))
secop_I_filter_norm_scra[['nombre_grupo', 'nombre_familia']] = secop_I_filter_norm_scra[['nombre_grupo','nombre_familia']].apply(lambda x: x.apply(norm_text))
secop_I_filter_norm_scra[['nombre_clase', 'tipo_identifi_del_contratista']] = secop_I_filter_norm_scra[['nombre_clase', 'tipo_identifi_del_contratista']].apply(lambda x: x.apply(norm_text))
secop_I_filter_norm_scra[['nom_razon_social_contratista','tipo_doc_representante_legal','moneda']] = secop_I_filter_norm_scra[['nom_razon_social_contratista', 'tipo_doc_representante_legal','moneda']].apply(lambda x: x.apply(norm_text))

* Limpio descripciones de proceso, dejando solo palabras claves

In [262]:
def clean_text(texto: str):
    # Convert text to lowercase
    texto = texto.lower()
    texto = texto.replace('\n', ' ') # remove line breaks
    texto = re.sub('\S*@\S*\s?', '', texto)  #remove emails 
    texto = re.sub(r'[0-9]+', '', texto) #remove numbers
    texto = strip_multiple_whitespaces(texto) #remove spaces
    word_tokens = simple_preprocess(texto, deacc=True, min_len=4, max_len=15)
    texto = ' '.join(word_tokens)
    return texto

In [263]:
process_text = lambda x: clean_text(x)
# Aplicamos la normalización de texto para descripción del proceso
secop_I_filter_norm_scra['detalle_del_objeto_a_contratar']= secop_I_filter_norm_scra['detalle_del_objeto_a_contratar'].astype(str)
secop_I_filter_norm_scra['detalle_del_objeto_a_contratar'] = secop_I_filter_norm_scra['detalle_del_objeto_a_contratar'].apply(process_text)
# Aplicamos la normalización de texto para objeto del contrato
secop_I_filter_norm_scra['objeto_del_contrato_a_la']= secop_I_filter_norm_scra['objeto_del_contrato_a_la'].astype(str)
secop_I_filter_norm_scra['objeto_del_contrato_a_la'] = secop_I_filter_norm_scra['objeto_del_contrato_a_la'].apply(process_text)
secop_I_filter_norm_scra.head(2)

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,171360274835540305,2017,13,contratacion minima cuantia,celebrado,50000000,suministro bebidas comestibles viveres rancho ...,suministro,acacias,acacias,meta,13017001,54408900,54408900,E,productos de uso final,5020,bebidas,502023,bebidas no alcoholicas,5540305,nit de persona juridica,860000258,compania manufacturera de pan comapan,cedula de ciudadania,41771248,30,D,0,0,3388200,0,0,3388200,peso colombiano,suministro comestibles viveres rancho destinad...,{'url': 'https://www.contratos.gov.co/consulta...
1,171373378886713176,2017,13,contratacion minima cuantia,celebrado,53000000,contratar adquisicion elementos seguridad indu...,compraventa,fusagasuga,fusagasuga,cundinamarca,0462017,284191,284191,E,productos de uso final,5310,ropa,531030,camisetas,6713176,nit de persona juridica,900988646,grupo ingeniar sas,cedula de ciudadania,35250339,15,D,0,0,270400,0,0,270400,peso colombiano,contratar adquisicion elementos seguridad indu...,{'url': 'https://www.contratos.gov.co/consulta...


* Remuevo los espacios de las columnas `numero_de_proceso` y `numero_de_contrato` ya que algunos valores con tienen letras y números, es decir, son valores alfanumericos.

In [268]:
def space_text(texto:str):
    texto = texto.replace(" ", "")
    return texto

In [269]:
# Aplicamos la normalización de texto para descripción del proceso
process_text = lambda x: space_text(x)

secop_I_filter_norm_scra['numero_de_proceso']= secop_I_filter_norm_scra['numero_de_proceso'].astype(str)
secop_I_filter_norm_scra['numero_de_proceso'] = secop_I_filter_norm_scra['numero_de_proceso'].apply(space_text)

secop_I_filter_norm_scra['numero_de_contrato']= secop_I_filter_norm_scra['numero_de_contrato'].astype(str)
secop_I_filter_norm_scra['numero_de_contrato'] = secop_I_filter_norm_scra['numero_de_contrato'].apply(space_text)
secop_I_filter_norm_scra.head(3)

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,171360274835540305,2017,13,contratacion minima cuantia,celebrado,50000000,suministro bebidas comestibles viveres rancho ...,suministro,acacias,acacias,meta,13017001,54408900,54408900,E,productos de uso final,5020,bebidas,502023,bebidas no alcoholicas,5540305,nit de persona juridica,860000258,compania manufacturera de pan comapan,cedula de ciudadania,41771248,30,D,0,0,3388200,0,0,3388200,peso colombiano,suministro comestibles viveres rancho destinad...,{'url': 'https://www.contratos.gov.co/consulta...
1,171373378886713176,2017,13,contratacion minima cuantia,celebrado,53000000,contratar adquisicion elementos seguridad indu...,compraventa,fusagasuga,fusagasuga,cundinamarca,0462017,284191,284191,E,productos de uso final,5310,ropa,531030,camisetas,6713176,nit de persona juridica,900988646,grupo ingeniar sas,cedula de ciudadania,35250339,15,D,0,0,270400,0,0,270400,peso colombiano,contratar adquisicion elementos seguridad indu...,{'url': 'https://www.contratos.gov.co/consulta...
2,151341535123918724,2015,13,contratacion minima cuantia,celebrado,53000000,adquisicion kits elementos aseo personal para ...,compraventa,bucaramanga,bucaramanga,santander,410043DE2515,52875000,52875000,E,productos de uso final,5313,articulos de tocador cuidado personal,531316,bano cuerpo,3918724,nit de persona juridica,8040037441,central de suministros ltda,cedula de ciudadania,28212971 de Lebrija,30,D,0,0,52872736,0,0,52872736,peso colombiano,adquisicion kits elementos aseo personal para ...,{'url': 'https://www.contratos.gov.co/consulta...


In [270]:
secop_I_filter_norm_scra.to_csv('data/extrac_secopI_final.csv', index=False)

#### Resumen estadístico
Comenzaremos analizando el resumen estadístico y luego creando diferentes gráficos y diagramas.

In [ ]:
#dd.data_summary(secop)